In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

# Function to create the custom DataFrame
def create_custom_df(start_date, end_date, days, nasdaq_indexes, market_caps, ew_index_data):
    # Convert start_date and end_date to datetime64[ns]
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter days DataFrame to the specified date range
    filtered_days = days[(days['Date'] >= start_date) & (days['Date'] <= end_date)].copy()

    # Initialize the custom DataFrame with the filtered days
    custom_df = pd.DataFrame(filtered_days['Date'], columns=['Date'])

    # Ensure the 'Date' column in the custom_df and all relevant DataFrames is datetime64[ns]
    custom_df['Date'] = pd.to_datetime(custom_df['Date'])
    
    # Add NASDAQ indexes data
    index_full_names = {
        '^IXIC': 'NASDAQ Composite',
        '^NDXE': 'NASDAQ 100 Equal Weighted',
        '^NDX': 'NASDAQ 100',
        '^GSPC': 'S&P 500',
        '^SPXEW': 'S&P 500 Equal Weight'
    }
    
    for index in nasdaq_indexes:
        index_data = yf.download(index, start=start_date, end=end_date)
        if 'Date' not in index_data.columns:
            index_data.reset_index(inplace=True)
        print(f"Merging Index {index_full_names[index]} with {len(index_data)} rows")
        custom_df = custom_df.merge(index_data[['Date', 'Close']], on='Date', how='left')
        custom_df.rename(columns={'Close': f'{index_full_names[index]}'}, inplace=True)

    # Add Overall Market Caps data
    market_caps['Date'] = pd.to_datetime(market_caps['Date'])
    overall_market_caps = market_caps[['Date', 'index']].copy()
    overall_market_caps = overall_market_caps.rename(columns={'index': 'Overall Market Caps'})
    print(f"Merging Overall Market Caps with {len(overall_market_caps)} rows")
    custom_df = custom_df.merge(overall_market_caps, on='Date', how='left')

    # Add Finder EW Index data
    ew_index_data['Date'] = pd.to_datetime(ew_index_data['Date'])
    finder_ew_index = ew_index_data[['Date', 'Finder EW Index']].copy()
    print(f"Merging Finder EW Index with {len(finder_ew_index)} rows")
    custom_df = custom_df.merge(finder_ew_index, on='Date', how='left')

    # Remove rows where all elements are NaN except the 'Date' column
    custom_df = custom_df.dropna(how='all', subset=custom_df.columns.difference(['Date']))

    return custom_df

# Function to normalize the DataFrame
def normalize_dataframe(df):
    normalized_df = df.copy()
    for column in df.columns:
        if column != 'Date':
            initial_index = df[column].first_valid_index()
            if initial_index is None:
                print(f"No valid data for {column}. Skipping normalization for this column.")
                normalized_df[column] = np.nan
            else:
                initial_value = df.at[initial_index, column]
                normalized_df[column] = (df[column] / initial_value) * 100
                normalized_df.at[initial_index, column] = 100  # Ensure the first valid value is set to 100
    print(f"Normalized DataFrame:\n{normalized_df.head()}")
    return normalized_df

# Load data from the provided Excel files
file_path = 'C:\\Users\\odeya.h\\SNC Dropbox\\Odeya Hazani Cohen\\פרוייקט\\calculate data.xlsx'
ew_index_file_path = 'C:\\Users\\odeya.h\\SNC Dropbox\\Odeya Hazani Cohen\\פרוייקט\\EW index data.xlsx'

xl = pd.ExcelFile(file_path)
ew_xl = pd.ExcelFile(ew_index_file_path)

# Read the Overall Market Caps sheet into a DataFrame
market_caps = xl.parse('Overall Market Caps_MC')

# Read the EW Index Data sheet into a DataFrame
ew_index_data = ew_xl.parse('EW index data.xlsx')

# Create a DataFrame with all dates
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2024, 6, 1)
days = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='D', name='Date'))
days['Date'] = pd.to_datetime(days['Date'])

# List of NASDAQ indexes
nasdaq_indexes = ['^IXIC', '^NDXE', '^NDX', '^GSPC', '^SPXEW']

# Create the custom DataFrame based on user input
custom_df = create_custom_df(start_date, end_date, days, nasdaq_indexes, market_caps, ew_index_data)

print(f"Custom DataFrame before normalization:\n{custom_df.head()}")
normalized_df = normalize_dataframe(custom_df)

if 'Overall Market Caps' in custom_df.columns:
    custom_df.rename(columns={'Overall Market Caps': 'Finder Index'}, inplace=True)
    normalized_df.rename(columns={'Overall Market Caps': 'Finder Index'}, inplace=True)

# Save the custom DataFrame to an Excel file
output_file_path = 'custom_data_indexes_only_with_full_names.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    custom_df.to_excel(writer, sheet_name='Custom Data', index=False)
    normalized_df.to_excel(writer, sheet_name='Normalized Data', index=False)

print(f"DataFrames have been saved to {output_file_path}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Merging Index NASDAQ Composite with 105 rows
Merging Index NASDAQ 100 Equal Weighted with 105 rows


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Merging Index NASDAQ 100 with 105 rows
Merging Index S&P 500 with 105 rows


[*********************100%%**********************]  1 of 1 completed


Merging Index S&P 500 Equal Weight with 105 rows
Merging Overall Market Caps with 1615 rows
Merging Finder EW Index with 1363 rows
Custom DataFrame before normalization:
        Date  NASDAQ Composite  NASDAQ 100 Equal Weighted    NASDAQ 100  \
1 2024-01-02      14765.940430                7063.359863  16543.939453   
2 2024-01-03      14592.209961                6968.740234  16368.490234   
3 2024-01-04      14510.299805                6943.680176  16282.009766   
4 2024-01-05      14524.070312                6950.770020  16305.980469   
7 2024-01-08      14843.769531                7078.250000  16649.869141   

       S&P 500  S&P 500 Equal Weight  Overall Market Caps  Finder EW Index  
1  4742.830078           6401.109863            72.040183      1228.391282  
2  4704.810059           6308.089844            70.346190      1213.183293  
3  4688.680176           6299.089844            66.929852      1211.710915  
4  4697.240234           6317.180176            67.010086      1210.305

In [18]:
# Save the custom DataFrame and normalized DataFrame to a new Excel file with a graph
output_path = 'C:\\Users\\odeya.h\\SNC Dropbox\\Odeya Hazani Cohen\\פרוייקט\\finder_index_data_2024.xlsx'
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    custom_df.to_excel(writer, sheet_name='Custom Data', index=False)
    normalized_df.to_excel(writer, sheet_name='Normalized Data', index=False)
    
    # Create a chart object
    workbook  = writer.book
    worksheet = writer.sheets['Normalized Data']
    chart = workbook.add_chart({'type': 'line'})

    # Configure the series of the chart from the DataFrame data
    for i, column in enumerate(normalized_df.columns[1:]):  # Skip the 'Date' column
        chart.add_series({
            'name':       [worksheet.name, 0, i + 1],
            'categories': [worksheet.name, 1, 0, len(normalized_df), 0],
            'values':     [worksheet.name, 1, i + 1, len(normalized_df), i + 1],
        })

    # Configure the chart title with professional formatting
    chart.set_title({
        'name': 'Index Comparison',
        'name_font': {
            'bold': True,
            'size': 14,
            'name': 'Arial'
        }
    })

    # Configure the chart axes with professional formatting and the new date format
    chart.set_x_axis({
        'name': 'Date',
        'date_axis': True,
        'num_format': 'dd-mm-yyyy',
        'name_font': {
            'bold': True,
            'size': 12,
            'name': 'Arial'
        },
        'label_font': {
            'size': 10,
            'name': 'Arial'
        },
        'major_gridlines': {
            'visible': True,
            'line': {'width': 0.75, 'dash_type': 'dash'}
        },
        'minor_gridlines': {
            'visible': True,
            'line': {'width': 0.25, 'dash_type': 'dash'}
        }
    })
    chart.set_y_axis({
        'name': 'Normalized Value',
        'name_font': {
            'bold': True,
            'size': 12,
            'name': 'Arial'
        },
        'label_font': {
            'size': 10,
            'name': 'Arial'
        },
        'major_gridlines': {
            'visible': True,
            'line': {'width': 0.75, 'dash_type': 'dash'}
        },
        'minor_gridlines': {
            'visible': True,
            'line': {'width': 0.25, 'dash_type': 'dash'}
        }
    })

    # Customize the legend
    chart.set_legend({
        'position': 'bottom',
        'font': {
            'size': 10,
            'name': 'Arial'
        }
    })

    # Adjust the chart size and positioning
    worksheet.insert_chart('G2', chart, {'x_offset': 25, 'y_offset': 10, 'x_scale': 1.5, 'y_scale': 1.5})

print(f"Custom data with graph has been saved to {output_path}")


Custom data with graph has been saved to C:\Users\odeya.h\SNC Dropbox\Odeya Hazani Cohen\פרוייקט\finder_index_data_2024.xlsx
